In [ ]:
import pandas as pd

#my_file = 

In [ ]:
df = pd.read_csv('test.csv', encoding='latin_1', skiprows=13, sep=';')

#print(df.to_string()) 

In [ ]:
new_df = df[["Buchung", "Auftraggeber/Empfänger", "Verwendungszweck", "Betrag"]]
new_df.rename(columns={
    "Buchung" : "Date",
    "Auftraggeber/Empfänger" : "Payee",
    "Verwendungszweck" : "Notes",
    "Betrag" : "Amount"
}, inplace=True)
print(new_df.to_string()) 

In [ ]:
for row in new_df.index:
    if new_df.loc[row, "Payee"].startswith('VISA'):
        new_df.at[row, "Payee"] =  new_df.loc[row, "Payee"][5:].title() # remove 'VISA ' from payee and turn UPPER to Capitalized string


In [ ]:
def remove_at(i, s):
    return s[:i] + s[i+1:]

for row in new_df.index:
    if new_df.loc[row, "Payee"] == 'PayPal Europe S.a.r.l. et Cie S.C.A':
        # exported csv from ing has a space at position 35 in the notes for paypal-transactions for some reason...
        if new_df.loc[row, "Notes"][35] == ' ':
            new_df.at[row, "Notes"] = remove_at(35, new_df.at[row, "Notes"])
        # replace paypal as payee with actual payee from notes
        pos_from = new_df.loc[row, "Notes"].find(' . ') + 3
        pos_to = new_df.loc[row, "Notes"].find(',')
        actual_payee = new_df.loc[row, "Notes"][pos_from:pos_to]
        new_df.at[row, "Payee"] = actual_payee


In [ ]:
s = "1025936808250 . Blizzard Entertainment SAS, Ihr Einkauf bei Blizzard E ntertainment SAS"
print(s.find(' . '))
print(s.find(','))
p = s[s.find(' . ')+3:s.find(',')]
print(p)

In [ ]:
print(new_df.to_string()) 

In [ ]:
new_df.to_csv(path_or_buf='output.csv', sep=';', index=False)